# Ex 2
Now let's combine what we have learnt here and in the "Dataset enrichment" notebook.
Let's create a network (with the features of the Dataset enrichment notebook one) and feed it with the data from the CIFAR-10 dataset (by using only 2-3 classes to save training time and see faster results).

Our goal in this exercise is to test if there is an effective improvment in using the operations for enriching the dataset. To do so we will:

1) Run the network by using the data from the CIFAR-10, without preprocessing, and store the network configuration that performs better on the test set after some epochs of training. To do so, write some code that 
- evaluate the accuracy on the test set every 100 iterations, and keep the maximum accuracy in memory. 
- As long as the network improves on the test set, save its current variables. 
- Stop the training phase when the network is not improving after 400 iterations. 

This technique is called "Early stopping and is used to prevent overfitting"

2) Run the same network, but instead of inizialiting it randomly, use the configuration of the previously trained network. This time, train the network with the enriched dataset and check wether the test prediction has improved.



In [1]:
import tensorflow as tf

def conv_weights(filters_size, channels_size, name):
    shape = filters_size + channels_size
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

def conv(x, W, stride, name):
    strides_shape = [1, stride, stride, 1]
    return tf.nn.conv2d(x, W, strides_shape, padding='SAME', name=name)

def pool(x, size, stride, name):
    pool_shape = [1] + size + [1]
    strides_shape = [1, stride, stride, 1]
    return tf.nn.max_pool(x, pool_shape, strides_shape, padding='SAME', name=name)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os
import cifar10
#Set it true if you want to retrieve data from the checkpoint
RESTORE_OPT = True 
ENRICHED_DATA = True

In [3]:
cifar10.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
from cifar10 import  num_channels
num_classes=4

In [6]:
upper_b = (len(class_names)-num_classes)
images_train, cls_train, labels_train = cifar10.load_training_data()
train_size= len(cls_train)
images_train, cls_train, labels_train = images_train[cls_train<num_classes], cls_train[cls_train<num_classes], labels_train[cls_train<num_classes]
images_test, cls_test, labels_test = cifar10.load_test_data()
test_size= len(cls_test)
images_test, cls_test, labels_test = images_test[cls_test<num_classes], cls_test[cls_test<num_classes], labels_test[cls_test<num_classes]
print("Size of:")
print("- Training-set:\t\t{}".format(len(images_train)))
print("- Test-set:\t\t{}".format(len(images_test)))

train_batch_size= 128
print (images_train.shape)
print (cls_train.shape)
print (labels_train.shape)


Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
Size of:
- Training-set:		20000
- Test-set:		4000
(20000, 32, 32, 3)
(20000,)
(20000, 10)


In [7]:
img_size = 32
img_size_cropped = 28

In [8]:
labels_train =  labels_train[0:train_size*num_classes/10,0:-upper_b]

labels_test =  labels_test[0:test_size*num_classes/10,0:-upper_b]

print (labels_train.shape)
print (labels_train)

(20000, 4)
[[ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 ..., 
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]]


/home/giovanni/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
/home/giovanni/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [9]:
def pre_process_image(image):
    image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, max_delta=0.5)
    image = tf.image.random_contrast(image, lower=0.03, upper=10.0)
    image = tf.image.random_brightness(image, max_delta=2)
    image = tf.image.random_saturation(image, lower=0.0, upper=20.0)
    #Some of these functions may produce an output value beyond the [0,1] range of an image
    #To prevent this effect, we limit the range 
    image = tf.minimum(image, 1.0)
    image = tf.maximum(image, 0.0)
    return image

In [10]:
def resize_util(image):
    image = tf.image.resize_image_with_crop_or_pad(image,
                                                   target_height=img_size_cropped,
                                                   target_width=img_size_cropped)
    image = tf.cast(image, tf.float32)
    return image

In [11]:
def pre_process(images):
    images = tf.map_fn(lambda image: pre_process_image(image), images)
    return images

In [12]:
def resize(images):
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn(lambda image: resize_util(image), images)
    return images

In [13]:
def random_batch(size, training= True):
    if training == True:
        num_images = len(images_train)
    else:
        num_images = len(images_test)
        
    # Create a random index.
    idx = np.random.choice(num_images,
                           size=size,
                           replace=False)
    # Use the random index to select random images and labels.
    if training == True:
        x_batch = images_train[idx, :, :, :]
        y_batch = labels_train[idx, :]
    else:
        x_batch = images_test[idx, :, :, :]
        y_batch = labels_test[idx, :]
        
    return x_batch.astype(np.float32), y_batch.astype(np.float32)

In [14]:
import math
BATCH_SIZE = 100
LEARNING_RATE = 0.0005 # BEST: GD Optimizer with lambda=0.5
ITERATIONS = 1000

input_ =tf.placeholder(tf.float32,[None, 32, 32, 3], name='input_images')
if ENRICHED_DATA:
    input_images = pre_process(images=input_)
else:
    input_images = resize(input_)
y_ = tf.placeholder(tf.float32, [None, num_classes], name='labels')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')

W1 = conv_weights([7, 7], [3, 32], 'L1_weights')
b1 = tf.Variable(tf.constant(0.1, shape=[32]), name='L1_biases')
c1 = conv(input_images, W1, stride=1, name='L1_conv')
h1 = tf.nn.relu(tf.nn.bias_add(c1, b1), name='L1_ReLU')
p1 = pool(h1, size=[2, 2], stride=2, name='L1_pool')
 
W2 = conv_weights([7,7], [32, 64], 'L2_weights')
b2 = tf.Variable(tf.constant(0.1, shape=[64]), name='L2_biases')
c2 = conv(p1, W2, stride=1, name='L2_conv')
h2 = tf.nn.relu(tf.nn.bias_add(c2, b2), name='L2_ReLU')
p2 = pool(h2, size=[2, 2], stride=2, name='L2_pool')


W3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=1 / math.sqrt(7 * 7 * 64)), name='L3_weights')
b3 = tf.Variable(tf.constant(0.1, shape=[1024]), name='L3_biases')
p2_flat = tf.reshape(p2, [-1, 7 * 7 * 64])
h3 = tf.nn.relu(tf.matmul(p2_flat, W3) + b3)
h3_drop = tf.nn.dropout(h3, 1.0 - drop_prob, name='L3_dropout')


W4 = tf.Variable(tf.truncated_normal([1024, num_classes], stddev=1 / math.sqrt(1024)), name='L4_weights')
b4 = tf.Variable(tf.constant(0.1, shape=[num_classes]), name='L4_biases')
y = tf.matmul(h3_drop, W4) + b4


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_), name='cross_entropy')
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='train_accuracy')


New model initialized


In [ ]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

In [18]:
save_dir = 'check/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if os.path.exists("check/checkpointerfre") and RESTORE_OPT:
    saver.restore(sess, "check/checkp")
    print ("Model restored")
else:
    print ("New model initialized")
    init = tf.global_variables_initializer()
    sess.run(init)


New model initialized


In [19]:
def train():
    max_acc = 0
    counter = 0
    for i in range(ITERATIONS):
        batch_xs, batch_ys = random_batch(BATCH_SIZE, training= True)
        _ = sess.run([train_step], feed_dict={input_: batch_xs, y_: batch_ys, drop_prob: np.float32(0.5)})
  
        if i % 50 == 0:
            acc = sess.run(accuracy, feed_dict={input_: batch_xs, y_: batch_ys, drop_prob: np.float32(0.0)})
            print('%d/%d. training accuracy = %.2f' % (i, ITERATIONS, acc))
        if i % 100 == 0:
            batch_xs_test, batch_ys_test = random_batch(len(images_test), training= False)
            test_acc = sess.run(accuracy, feed_dict={input_: batch_xs_test, y_: batch_ys_test,drop_prob: np.float32(0.0)})
            counter = counter + 100
            if counter >500:
                return "The test accuracy has stopped increasing"
            else:
                if test_acc > max_acc:
                    max_acc = test_acc
                    counter = 0
                    save_path = saver.save(sess, "check/checkp")
                    print("Model saved in file: %s" % save_path)
            print('test accuracy = %.2f' % test_acc)




In [20]:
train()

0/1000. training accuracy = 0.33
Model saved in file: check/checkp
test accuracy = 0.25
50/1000. training accuracy = 0.38
100/1000. training accuracy = 0.39
Model saved in file: check/checkp
test accuracy = 0.44
150/1000. training accuracy = 0.51
200/1000. training accuracy = 0.48
Model saved in file: check/checkp
test accuracy = 0.48
250/1000. training accuracy = 0.38
300/1000. training accuracy = 0.53
test accuracy = 0.47
350/1000. training accuracy = 0.54
400/1000. training accuracy = 0.48
Model saved in file: check/checkp
test accuracy = 0.52
450/1000. training accuracy = 0.53
500/1000. training accuracy = 0.59
test accuracy = 0.52
550/1000. training accuracy = 0.57
600/1000. training accuracy = 0.56
Model saved in file: check/checkp
test accuracy = 0.55
650/1000. training accuracy = 0.56
700/1000. training accuracy = 0.52
Model saved in file: check/checkp
test accuracy = 0.56
750/1000. training accuracy = 0.58
800/1000. training accuracy = 0.57
Model saved in file: check/checkp
te